In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import pyGadget
import matplotlib as mpl
from matplotlib import pyplot as plt

In [ ]:
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
sns.despine()
mpl.rc('text', usetex=True)

# Halo Mass Growth History

In [ ]:
simname = {'vanilla':r'$u_{\rm \tiny CR} = 0$',
        'cr00': r'$u_{\rm \tiny CR} = u_0$', 
        'cr01': r'$u_{\rm \tiny CR} = 10 u_0$', 
        'cr02': r'$u_{\rm \tiny CR} = 10^2 u_0$', 
        'cr03': r'$u_{\rm \tiny CR} = 10^3 u_0$', 
        'cr04': r'$u_{\rm \tiny CR} = 10^4 u_0$',
        'cr05': r'$u_{\rm \tiny CR} = 10^5 u_0$'}
simlist = ['cr00','cr01','cr02','cr03','cr04','cr05']
name = ['vanilla', 'cr00', 'cr01', 'cr02', 'cr03', 'cr04', 'cr05']

In [ ]:
store = pd.HDFStore(os.getenv("HOME")+'/data/stampede/mass_history.hdf5')
mg = store['vanilla']#.set_index('time')
mg0 = store['cr00']#.set_index('time')
mg1 = store['cr01']#.set_index('time')
mg2 = store['cr02']#.set_index('time')
mg3 = store['cr03']#.set_index('time')
mg4 = store['cr04']#.set_index('time')
mg5 = store['cr05']#.set_index('time')
store.close()

In [ ]:
store = pd.HDFStore(os.getenv("HOME")+'/data/stampede/sinkdata.hdf5')
sinksV = store['vanilla']
sinks0 = store['cr00']
sinks1 = store['cr01']
sinks2 = store['cr02']
sinks3 = store['cr03']
sinks4 = store['cr04']
sinks5 = store['cr05']
store.close()

In [ ]:
sinksV.groupby('ID').last().describe()

In [ ]:
data = {'vanilla':mg, 'cr00':mg0, 'cr01':mg1, 'cr02':mg2, 'cr03':mg3, 'cr04':mg4, 'cr05':mg5}
sinkdata = {'vanilla':sinksV, 'cr00':sinks0, 'cr01':sinks1, 'cr02':sinks2, 'cr03':sinks3, 'cr04':sinks4, 'cr05':sinks5}

In [ ]:
for sim in data.keys():
    df = data[sim]
    print sim, df[df.time > 0].z.iloc[0]

In [ ]:
total_mass = {}
for i, sim in enumerate(data.keys()):
    mtot = sinkdata[sim][['time','mass']].groupby('time').sum()
    mtot.rename(columns = {'mass':simname[sim]}, inplace=True)
    total_mass[sim] = mtot.reset_index()

In [ ]:
combined = {}
for i, sim in enumerate(name):
    df = data[sim]
    sdf = total_mass[sim].rename(columns = {simname[name[i]]:'sinks'})
    cdf = pd.merge(df, sdf, on='time')
    cdf['SFE'] = cdf['sinks'] / cdf['100cc']
    cdf["DFE"] = cdf['1e8cc'] / cdf['100cc']
    cdf['AcE'] = cdf['sinks'] / cdf['1e8cc']
    combined[sim] = cdf
#    df['SFE'] = df['100cc'] / 

In [ ]:
nkeys = ['10cc', '100cc', '1e4cc', '1e8cc', '1e10cc']
rkeys = ['100pc', '10pc', '1pc', '.1pc', '1e4AU', '5e3AU', '1e3AU']

In [ ]:
fig, axarr = plt.subplots(3, sharex=True, figsize=(9,15))

for i, dlim in enumerate(nkeys[2:]):
    axarr[i].axvline(0, lw=1, ls='-', c='.8')
    axarr[i] = mg[mg[dlim] > 0].plot(ax=axarr[i], x='time', y=dlim, label=simname['vanilla'],
                                    ls='-', color='.25', grid=False)
    for sim in simlist:
        df = data[sim]
        axarr[i] = df[df[dlim] > 0].plot(ax=axarr[i], x='time', y=dlim, label=simname[sim])#, grid=False)
    axarr[i].set_ylabel('Mass [M$_{\odot}$]')
    axarr[i].set_xlabel("")
    axarr[i].text(.97,.03,"$n >$ "+dlim.replace('cc', ' cm$^{-3}$') , ha='right', va='bottom', 
                  size=20, transform=axarr[i].transAxes)

axarr[0].set_ylim(500, 1.4e3)
axarr[1].set_ylim(0, 130)
axarr[2].set_ylim(0, 75)
axarr[1].legend(loc=0)
axarr[2].set_xlabel("Time since sink formation [yr]")
plt.xlim(-5e3, 5e3)
fig.subplots_adjust(hspace=0.0)
sns.despine()
#plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)

In [ ]:
with sns.axes_style("darkgrid"):
    fig, ax = plt.subplots()
    df = data['vanilla']
    df[df['1e8cc'] > 0].plot(ax=ax, x='time', y='1e8cc', color='.25', label=simname['vanilla'])
    for sim in simlist:
        df = data[sim]
        df[df['1e8cc'] > 0].plot(ax=ax, x='time', y='1e8cc', label=simname[sim])
    ax.set_xlim(-7500,5000)
    ax.set_ylim(0,120)
    ax.legend(loc=0)
    ax.set_ylabel('Mass [M$_{\odot}$]')
    ax.set_xlabel("Time since sink formation [yr]")

In [ ]:
with sns.axes_style("white"):
    fig, ax = plt.subplots()
df = data['vanilla']
df = df[df['1pc'] > 0]
ax.semilogy(df.z, df['1pc'], color='.25', label=simname['vanilla'])
for sim in simlist[1:]:
    df = data[sim]
    df = df[df['1pc'] > 0]
    ax.plot(df.z, df['1pc'], label=simname[sim])
ax.set_color_cycle(None)
ax.set_xlim(34, 25)
ax.set_ylim(2,2e3)
ax.legend(loc=0)
ax.set_ylabel('Mass within 1 pc [M$_{\odot}$]')
ax.set_xlabel("Redshift")
fig.savefig('figures/growth/collapse.png', bbox_inches='tight')
#fig.savefig('figures/growth/collapse.pdf', bbox_inches='tight')

In [ ]:
fig, axarr = plt.subplots(3, sharex=True, figsize=(9,15))

for i, dlim in enumerate(nkeys[:3]):
    axarr[i].axvline(0, lw=1, ls='-', c='.8')
    axarr[i] = mg[mg[dlim] > 0].plot(ax=axarr[i], x='z', y=dlim, label=simname['vanilla'],
                                    ls='-', color='.25', grid=False, logy=True)
    for sim in simlist:
        df = data[sim]
        axarr[i] = df[df[dlim] > 0].plot(ax=axarr[i], x='z', y=dlim, label=simname[sim], logy=True)#, grid=False)
    axarr[i].set_ylabel('Mass [M$_{\odot}$]')
    axarr[i].set_xlabel("")
    axarr[i].text(.5,.95,"$n >$ "+dlim.replace('cc', ' cm$^{-3}$') , ha='center', va='top', 
                  size=20, transform=axarr[i].transAxes)

axarr[0].set_ylim(1, 1e4)
axarr[1].set_ylim(.2, 5e3)
axarr[2].set_ylim(.2, 5e3)
axarr[1].legend(loc=0)
axarr[2].set_xlabel("Redshift")
plt.xlim(42, 25)
fig.subplots_adjust(hspace=0.0)
sns.despine()
#plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)

In [ ]:
fig, axarr = plt.subplots(3, sharex=True, figsize=(8,15))

for i, dlim in enumerate(rkeys[:3]):
    axarr[i].axvline(0, lw=1, ls='-', c='.8')
    axarr[i] = mg[mg[dlim] > 0].plot(ax=axarr[i], x='z', y=dlim, label=simname['vanilla'],
                                    ls='-', color='.25', grid=False, logy=False)
    for sim in simlist:
        df = data[sim]
        axarr[i] = df[df[dlim] > 0].plot(ax=axarr[i], x='z', y=dlim, label=simname[sim], logy=False)#, grid=False)
    axarr[i].set_ylabel('Mass [M$_{\odot}$]')
    axarr[i].set_xlabel("")
    axarr[i].text(.5,.95,"$r <$ "+dlim, ha='center', va='top', 
                  size=20, transform=axarr[i].transAxes)

#axarr[0].set_ylim(1, 1e4)
#axarr[1].set_ylim(.2, 5e3)
#axarr[2].set_ylim(.2, 5e3)
axarr[1].legend(loc=0)
axarr[2].set_xlabel("Redshift")
plt.xlim(35, 24.95)
fig.subplots_adjust(hspace=0.003)
sns.despine()
fig.savefig('figures/growth/massgrowth_lores.png', bbox_inches='tight')

In [ ]:
fig, axarr = plt.subplots(3, sharex=True, figsize=(9,15))

for i, dlim in enumerate([rkeys[i] for i in [3,4,5]]):
    axarr[i].axvline(0, lw=1, ls='-', c='.8')
    axarr[i] = mg[mg[dlim] > 0].plot(ax=axarr[i], x='time', y=dlim, label=simname['vanilla'],
                                    ls='-', color='.25', grid=False)
    for sim in simlist:
        df = data[sim]
        axarr[i] = df[df[dlim] > 0].plot(ax=axarr[i], x='time', y=dlim, label=simname[sim])#, grid=False)
    axarr[i].set_ylabel('Mass [M$_{\odot}$]')
    axarr[i].set_xlabel("")
    axarr[i].text(.97,.03,"$r <$ "+dlim , ha='right', va='bottom', 
                  size=20, transform=axarr[i].transAxes)

axarr[0].set_ylim(125, 450)
axarr[1].set_ylim(75, 275)
axarr[2].set_ylim(25, 190)
axarr[2].legend(loc=0)
axarr[2].set_xlabel("Time since sink formation [yr]")
plt.xlim(-5e3, 5e3)
fig.subplots_adjust(hspace=0.0)
sns.despine()
#plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)

In [ ]:
eff_label = {'SFE':"Star Formation Efficiency", 'DFE':"Disk Formation Efficiency", 'AcE':"Sink Accretion Efficiency"}

In [ ]:
eff = 'SFE'
with sns.axes_style("darkgrid"):
    fig, ax = plt.subplots()
    df = combined['vanilla'].set_index('time')
    ax = df[eff].plot(ax=ax, label=simname['vanilla'], ls='-', color='.25')#, grid=False)
    for sim in simlist:
        df = combined[sim].set_index('time')
        ax = df[eff].plot(ax=ax, label=simname[sim])#, grid=False)
    ax.set_ylabel(eff_label[eff])
    ax.set_xlim(0,5000)
    ax.set_ylim(0,.04)
    ax.legend(loc=2)
#sns.despine()
fig.savefig('figures/growth/efficiency.png', bbox_inches='tight')

In [ ]:
with sns.axes_style("darkgrid"):
    fig, axl = plt.subplots(1,2, sharex=True, figsize=(15,6))
    eff_label = {'SFE':"Star Formation Efficiency", 'DFE':"Disk Formation Efficiency", 'AcE':"Sink Accretion Efficiency"}
    for i, eff in enumerate(['SFE', 'DFE']):
        df = combined['vanilla'].set_index('time')
        axl[i] = df[eff].plot(ax=axl[i], label=simname['vanilla'], ls='-', color='.25')#, grid=False)
        for sim in simlist:
            df = combined[sim].set_index('time')
            axl[i] = df[eff].plot(ax=axl[i], label=simname[sim])#, grid=False)
        axl[i].set_ylabel(eff_label[eff])
    axl[0].set_xlim(0,5000)
    axl[0].set_ylim(0,.035)
    axl[1].set_ylim(0,0.05)#1.005)
    axl[0].legend(loc=2)
#sns.despine()
fig.savefig('figures/growth/efficiency.png', bbox_inches='tight')

In [ ]:
fig, ((ax0,ax1),(ax2,ax3)) = plt.subplots(2,2, figsize=(12,10))

for ax in fig.axes:
    ax.axvline(0, lw=1.2, ls='-', c='.75')
    
ax0 = mg.plot(ax=ax0, x='time', y='100cc', ls='-', color='.25')
ax1 = mg.plot(ax=ax1, x='time', y='1e8cc', ls='-', color='.25', label=simname['vanilla'])
ax2 = combined['vanilla'].plot(ax=ax2, x='time', y='SFE', ls='-', color='.25')
ax3 = total_mass['vanilla'].plot(ax=ax3, x='time', y=simname['vanilla'], ls='-', color='.25')

#============ Panel A ==============
for sim in simlist:
    df = data[sim]
    ax0 = df.plot(ax=ax0, x='time', y='100cc', label=simname[sim])
ax0.set_ylim(1900,2700)
ax0.set_ylabel('$n >$ 100 cm$^{-3}$ Mass [M$_{\odot}$]')

#============ Panel B ==============
for sim in simlist:
    df = data[sim]
    ax1 = df.plot(ax=ax1, x='time', y='1e8cc', label=simname[sim])
ax1.set_ylim(0,112)
ax1.set_ylabel('$n > 10^8$ cm$^{-3}$ Mass [M$_{\odot}$]')
ax1.legend(loc=0)

#============ Panel C ==============
for sim in simlist:
    df = combined[sim]
    ax2 = df.plot(ax=ax2, x='time', y="SFE", label=simname[sim])
ax2.set_ylim(0,.035)
ax2.set_ylabel('Star Formation Efficiency')

#============ Panel D ==============
for sim in simlist:
    df = total_mass[sim]
    ax3 = df.plot(ax=ax3, x='time', y=simname[sim], label=simname[sim])
ax3.set_ylim(0,70)
ax3.set_ylabel('Total Sink Mass [M$_{\odot}$]')


for ax in fig.axes:
    ax.set_xlim(-1.8e3,5e3)
    ax.set_xticks((-1e3, 0, 1e3, 3e3, 5e3))
    ax.set_xlabel('Time Since Sink Formation [yr]')
#fig.subplots_adjust(vspace=1)
fig.tight_layout()
fig.savefig('figures/growth/sinkgrowth.png', bbox_inches='tight')